In [15]:
pip install os

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement os (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for os


In [ ]:
import pandas as pd

Import danych: New York City Airbnb - zbiór danych opisuje aktywność i wskaźniki ofert na Airbnb w Nowym Jorku w 2019 roku.

In [16]:
import kagglehub
path = kagglehub.dataset_download('dgomonov/new-york-city-airbnb-open-data')
print('Ścieżka do zbioru danych:',path)

Ścieżka do zbioru danych: C:\Users\majal\.cache\kagglehub\datasets\dgomonov\new-york-city-airbnb-open-data\versions\3


In [51]:
import os
directory_name = 'projekt_analiza_danych_python'
os.mkdir(directory_name)
file_path = "AB_NYC_2019.csv" 
dane = pd.read_csv(file_path)

FileExistsError: [WinError 183] Nie można utworzyć pliku, który już istnieje: 'projekt_analiza_danych_python'

Przegląd danych

In [52]:
dane.info()
print('Nasza tabela danych zawiera 48895 wierszy oraz 16 kolumn, \nw których dane przechowywane są w typach: int (liczby całkowite), float (liczby rzeczywiste) oraz object (objekty)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

Data wrangling. Do przeprowadzenia kompletnej analizy naszej bazy danych potrzebujemy jej w formie, tzw. czystej. Dlatego musimy najpierw nadać odpowiedni kształt danym, ocenić braki, obserwacje ostające czy też zlokalizować i usunąć błędy.

1. Braki danych

In [60]:
braki_danych = dane.isna().sum()
print(braki_danych)
print('Kolumny last_review i reviews_per_month mogą być puste, ale tylko dla wartosci 0 w kolumnie number_of_reviews')

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64
Kolumny last_review i reviews_per_month mogą być puste, ale tylko dla wartosci 0 w kolumnie number_of_reviews


In [75]:
for index, row in dane.iterrows():
    if row['number_of_reviews']==0 and ((not pd.isna(row['reviews_per_month'])) or (not pd.isna(row['last_review']))):
        print(row)

print('Warunek spełniony, braki danych są uzasadnione')


Warunek spełniony, braki danych są uzasadnione


In [65]:
spr_zalozenia_na = dane['number_of_reviews']==0 & ((~dane['last_review'].isna()) | (~dane['reviews_per_month'].isna()))
print(spr_zalozenia_na)

0        False
1        False
2         True
3        False
4        False
         ...  
48890     True
48891     True
48892     True
48893     True
48894     True
Length: 48895, dtype: bool


2. Typy zmiennych

In [91]:
# id i host_id - tekst
dane['id']=str(dane['id'])
dane['host_id']=str(dane['host_id'])

# room_type - skala porządkowa
kat_room_type = ['Shared room','Private room','Entire home/apt']
dane['room_type'] = pd.Categorical(dane['room_type'], categories=kat_room_type, ordered=True)

# last_review - data
dane['last_review']=pd.to_datetime(dane['last_review'])

dane.dtypes


0   2018-10-19
1   2019-05-21
2          NaT
3   2019-07-05
4   2018-11-19
Name: last_review, dtype: datetime64[ns]


id                                        object
name                                      object
host_id                                   object
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                               category
price                                      int64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
dtype: object